In [3]:
import pandas as pd
from product.serializers.product import CreateProductRequestGrpcSerializer
from product.viewsets.product import _upsert_product
from django.contrib.auth.models import User
from alloff_backoffice_server.settings import SERVICE_ENV_IS_DEV

df = pd.read_csv("allProducts.csv", sep=";").astype(str)
reg_df = pd.read_csv("allProducts-reg.csv")
df.set_index("Product Url", inplace=True)
reg_df.set_index("링크", inplace=True)
df = df[df.index.isin(reg_df.index)]

EURKRW = 1350
GROUPDEAL_IMG = "https://alloff.s3.ap-northeast-2.amazonaws.com/images/4HZCvH_1649490299_Untitled__RESIZE-mw1280.webp"
GROUPDEAL_DESC_IMG = "https://alloff.s3.ap-northeast-2.amazonaws.com/images/37XBBZ_1649410917_%E1%84%80%E1%85%B3%E1%84%85%E1%85%AE%E1%86%B8%E1%84%83%E1%85%B5%E1%86%AF%20-%20%E1%84%89%E1%85%A5%E1%86%AF%E1%84%86%E1%85%A7%E1%86%BC__RESIZE-mw1280.webp"
CATEGORY_ID = "621008161c35be7a91ccbfb7"

INFO_TRANSLATION = {
    "Bag length": "백 길이",
    "Bag height": "백 높이",
    "Bag weight": "백 무게",
    "Handle height": "핸들 높이",
    "Shoulder bag length": "숄더백 길이",
    "Belt length": "벨트 길이",
    "Belt height": "벨트 높이",
    "Accessory length": "악세서리 길이",
    "Accessory height": "악세서리 높이",
    "Accessory weight": "악세서리 무게",
    "Heel height": "힐 높이",
    "Plateau height": "높이",
    "Insole length": "안감 길이",
}


user = User.objects.first()
print(f"USER: {user}", " /// " f"ENV IS DEV: {SERVICE_ENV_IS_DEV}")

USER: admin  /// ENV IS DEV: False


In [9]:
def make_product_request(row):
    op = int(float(row["Retail Price"].replace(",", "").split(" ")[-1]))
    dr = float(row["Discount"].replace("%", "")) / 100
    
    sizes = row.Size.split(",")
    quantities = row["Qty Detail"].split(",")
    
    inventories = []
    for i in range(len(sizes)):
        s = sizes[i]
        q = int(quantities[i])
        if q > 0:
            inventories.append({
                "size": s,
                "quantity": q,
            })
    
    images = [x for x in [row.Image, row.Image1, row.Image2, row.Image3] if not pd.isnull(x) and x != "nan"]
    desc_images = [GROUPDEAL_IMG, GROUPDEAL_DESC_IMG]
    infos = {
            "소재": row.Material,
            "색상": row["Color detail"],
        }
    
    for I in INFO_TRANSLATION.keys():
        i = row[I]
        if i != "0":
            infos[INFO_TRANSLATION[I]] = i
    
    
    return {
        "alloff_name": f"[그룹딜2] {row.Name}",
        "is_foreign_delivery": True,
        "product_id": row["Sku Supplier"],
        "product_url": row.name,
        "original_price": int(op) * EURKRW,
        "discounted_price": int(op * dr) * EURKRW,
        "special_price": None,
        "brand_key_name": reg_df.loc[row.name].brand_keyname,
        "inventory": inventories,
        "description": [row.Description],
        "is_refund_possible": True,
        "images": images,
        "description_images": desc_images,
        "earliest_delivery_days": 14,
        "latest_delivery_days": 21,
        "refund_fee": 100_000,
        "alloff_category_id": CATEGORY_ID,
        "description_infos": infos,
        "product_infos": {
            "생산국가": row["Made in"]
        },
        "thumbnail_image": images[0],
    }

In [10]:
for i, x in df.iterrows():
    print("trying: ", x.name)
    req = make_product_request(x)
    s = CreateProductRequestGrpcSerializer(data=req)
    s.is_valid(raise_exception=True)
    s.user = user
    _upsert_product(s, is_update=False)
    print("done!")

trying:  https://www.styleisnow.com/business/maglie-prada-211197dpl000001-f0031.html
done!
trying:  https://www.styleisnow.com/business/maglie-kenzo-211416dma000003-76n.html
done!
trying:  https://www.styleisnow.com/business/maglie-loewe-211008dpl000001-2370.html
done!
trying:  https://www.styleisnow.com/business/t-shirts-e-polo-chloe-211512dts000004-81r.html
done!
trying:  https://www.styleisnow.com/business/top-chloe-211512dto000003-101.html
done!
trying:  https://www.styleisnow.com/business/maglie-s-max-mara-211718dma000003-002er.html
done!
trying:  https://www.styleisnow.com/business/maglie-s-max-mara-211718dma000003-009lu.html
done!
trying:  https://www.styleisnow.com/business/borse-a-p-c-212633abs000001-lzz.html
done!
trying:  https://www.styleisnow.com/business/t-shirts-e-polo-a-p-c-212633dts000003-aab.html
done!
trying:  https://www.styleisnow.com/business/t-shirts-e-polo-a-p-c-212633dts000003-cag.html
done!
trying:  https://www.styleisnow.com/business/borse-marni-221418abs0000